In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
import pickle

In [17]:
def read_data(filename, pickup_time_column, dropoff_time_column, filter_data=True):
    df = pd.read_parquet(filename)
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['duration'] = pd.to_datetime(df[dropoff_time_column]) - pd.to_datetime(df[pickup_time_column])
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    if filter_data:
        df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    return df

In [ ]:
def pre_process_data(df):
    
    df['PO_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    categorical = ['PO_DO'] #['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    x_dict = df[categorical + numerical].to_dict(orient='records')
    
    target = 'duration'
    y = df[target].values
    
    return x_dict, y

In [ ]:
dv = DictVectorizer()

df_train = read_data('../data/yellow_tripdata_2021-01.parquet', 'lpep_pickup_datetime', 'lpep_dropoff_datetime')
train_dict, y_train = pre_process_data(df_train)
x_train = dv.fit_transform(train_dict)

df_val = read_data('../data/yellow_tripdata_2021-02.parquet', 'lpep_pickup_datetime', 'lpep_dropoff_datetime')
val_dict, y_val = pre_process_data(df_val)
x_val = dv.transform(val_dict)

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_val)
sns.distplot(y_val, label='actual')
sns.distplot(y_pred, label='prediction')
plt.legend()

In [ ]:
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
mean_squared_error(y_train, y_pred, squared=False)

In [ ]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

## Homework Code

In [15]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')
len(df.columns)

19

In [18]:
df = read_data(
    '../data/yellow_tripdata_2023-01.parquet',
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    filter_data=False)
df.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [19]:
filtered_df = read_data(
    '../data/yellow_tripdata_2023-01.parquet',
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime')
len(filtered_df) / len(df)

0.9812202822125979

In [27]:
categorical = ['PULocationID', 'DOLocationID']
features = filtered_df[categorical].to_dict(orient='records')
dv = DictVectorizer()
features = dv.fit_transform(features)
features

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [28]:
labels = filtered_df.duration
lr = LinearRegression()
lr.fit(features, labels)
y_pred = lr.predict(features)
mean_squared_error(labels, y_pred, squared=False)

7.6492610279057605

In [ ]:
df_val = read_data(
    '../data/yellow_tripdata_2023-02.parquet',
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime')
features_val = df_val[categorical].to_dict(orient='records')
features_val = dv.transform(features_val)
labels_val = df_val.duration
lr.predict(features_val)
y_val = lr.predict(features)
mean_squared_error(labels_val, y_val, squared=False)

In [25]:
features

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>